<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# LightGCN - simplified GCN model for recommendation

This notebook serves as an introduction to LightGCN [1], which is an simple, linear and neat Graph Convolution Network (GCN) [3] model for recommendation.

## 0 Global Settings and Imports

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from datasets.outfits import load_pandas_df
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
# from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 18:02:02) 
[Clang 18.1.8 ]
Pandas version: 2.3.3
Tensorflow version: 2.14.0


/opt/homebrew/Caskroom/miniforge/base/envs/ai_ml-ncf/lib/python3.9/site-packages/pandera/_pandas_deprecated.py:160: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [2]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 50
BATCH_SIZE = 16

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "./recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

## 1 LightGCN model

LightGCN is a simplified version of Neural Graph Collaborative Filtering (NGCF) [4], which adapts GCNs in recommendation systems.

### 1.1 Graph Networks in Recommendation Systems

GCN are networks that can learn patterns in graph data. They can be applied in many fields, but they are particularly well suited for Recommendation Systems, because of their ability to encode relationships.

In traditional models like matrix factorization [5], user and items are represented as embeddings. And the interaction, which is the signal that encodes the behavior, is not part of the embeddings, but it is represented in the loss function, typically as a dot product. 

Despite their effectiveness, some authors [1,4] argue that these methods are not sufficient to yield satisfactory embeddings for collaborative filtering. The key reason is that the embedding function lacks an explicit encoding of the crucial collaborative signal, which is latent in user-item interactions to reveal the behavioral similarity between users (or items). 

**GCNs can be used to encode the interaction signal in the embeddings**. Interacted items can be seen as user´s features, because they provide direct evidence on a user’s preference. Similarly, the users that consume an item can be treated as the item’s features and used to measure the collaborative similarity of two items. A natural way to incorporate the interaction signal in the embedding is by exploiting the high-order connectivity from user-item interactions.

In the figure below, the user-item interaction is shown (to the left) as well as the concept of higher-order connectivity (to the right).

<img src="https://recodatasets.z20.web.core.windows.net/images/High_order_connectivity.png" width=500 style="display:block; margin-left:auto; margin-right:auto;">

The high-order connectivity shows the collaborative signal in a graph form. For example, the path $u_1 ← i_2 ← u2$ indicates the behavior
similarity between $u_1$ and $u_2$, as both users have interacted with $i_2$; the longer path $u_1 ← i_2 ← u_2 ← i_4$ suggests that $u_1$ is likely to adopt $i_4$, since her similar user $u_2$ has consumed $i_4$ before. Moreover, from the holistic view of $l = 3$, item $i_4$ is more likely to be of interest to $u_1$ than item $i_5$, since there are two paths connecting $(i_4,u_1)$, while only one path connects $(i_5,u_1)$.

Based on this high-order connectivity, NGCF [4] defines an embedding propagation layer, which refines a user’s (or an item’s) embedding by aggregating the embeddings of the interacted items (or users). By stacking multiple embedding propagation layers, we can enforce the embeddings
to capture the collaborative signal in high-order connectivities.

More formally, let $\mathbf{e}_{u}^{0}$ denote the original embedding of user $u$ and $\mathbf{e}_{i}^{0}$ denote the original embedding of item $i$. The embedding propagation can be computed recursively as:

$$
\begin{array}{l}
\mathbf{e}_{u}^{(k+1)}=\sigma\bigl( \mathbf{W}_{1}\mathbf{e}_{u}^{(k)} + \sum_{i \in \mathcal{N}_{u}} \frac{1}{\sqrt{\left|\mathcal{N}_{u}\right|} \sqrt{\left|\mathcal{N}_{i}\right|}} (\mathbf{W}_{1}\mathbf{e}_{i}^{(k)} + \mathbf{W}_{2}(\mathbf{e}_{i}^{(k)}\cdot\mathbf{e}_{u}^{(k)}) ) \bigr)
\\
\mathbf{e}_{i}^{(k+1)}=\sigma\bigl( \mathbf{W}_{1}\mathbf{e}_{i}^{(k)} +\sum_{u \in \mathcal{N}_{i}} \frac{1}{\sqrt{\left|\mathcal{N}_{i}\right|} \sqrt{\left|\mathcal{N}_{u}\right|}} (\mathbf{W}_{1}\mathbf{e}_{u}^{(k)} + \mathbf{W}_{2}(\mathbf{e}_{u}^{(k)}\cdot\mathbf{e}_{i}^{(k)}) ) \bigr)
\end{array}
$$

where $\mathbf{W}_{1}$ and $\mathbf{W}_{2}$ are trainable weight matrices, $\frac{1}{\sqrt{\left|\mathcal{N}_{i}\right|} \sqrt{\left|\mathcal{N}_{u}\right|}}$ is a discount factor expressed as the graph Laplacian norm, $\mathcal{N}_{u}$ and $\mathcal{N}_{i}$ denote the first-hop neighbors of user $u$ and item $i$, and $\sigma$ is a non-linearity that in the paper is set as a LeakyReLU. 

To obtain the final representation, each propagated embedding is concatenated (i.e., $\mathbf{e}_{u}^{(*)}=\mathbf{e}_{u}^{(0)}||...||\mathbf{e}_{u}^{(l)}$), and then the final user's preference over an item is computed as a dot product: $\hat y_{u i} = \mathbf{e}_{u}^{(*)T}\mathbf{e}_{i}^{(*)}$.

### 1.2 LightGCN architecture

LightGCN is a simplified version of NGCF [4] to make it more concise and appropriate for recommendations. The model architecture is illustrated below.

<img src="https://recodatasets.z20.web.core.windows.net/images/lightGCN-model.jpg" width=600 style="display:block; margin-left:auto; margin-right:auto;">

In Light Graph Convolution, only the normalized sum of neighbor embeddings is performed towards next layer; other operations like self-connection, feature transformation via weight matrices, and nonlinear activation are all removed, which largely simplifies NGCF. In the layer combination step, instead of concatenating the embeddings, we sum over the embeddings at each layer to obtain the final representations.

### 1.3 Light Graph Convolution (LGC)

In LightGCN, we adopt the simple weighted sum aggregator and abandon the use of feature transformation and nonlinear activation. The graph convolution operation in LightGCN is defined as:

$$
\begin{array}{l}
\mathbf{e}_{u}^{(k+1)}=\sum_{i \in \mathcal{N}_{u}} \frac{1}{\sqrt{\left|\mathcal{N}_{u}\right|} \sqrt{\left|\mathcal{N}_{i}\right|}} \mathbf{e}_{i}^{(k)} \\
\mathbf{e}_{i}^{(k+1)}=\sum_{u \in \mathcal{N}_{i}} \frac{1}{\sqrt{\left|\mathcal{N}_{i}\right|} \sqrt{\left|\mathcal{N}_{u}\right|}} \mathbf{e}_{u}^{(k)}
\end{array}
$$

The symmetric normalization term $\frac{1}{\sqrt{\left|\mathcal{N}_{u}\right|} \sqrt{\left|\mathcal{N}_{i}\right|}}$ follows the design of standard GCN, which can avoid the scale of embeddings increasing with graph convolution operations.


### 1.4 Layer Combination and Model Prediction

In LightGCN, the only trainable model parameters are the embeddings at the 0-th layer, i.e., $\mathbf{e}_{u}^{(0)}$ for all users and $\mathbf{e}_{i}^{(0)}$ for all items. When they are given, the embeddings at higher layers can be computed via LGC. After $K$ layers LGC, we further combine the embeddings obtained at each layer to form the final representation of a user (an item):

$$
\mathbf{e}_{u}=\sum_{k=0}^{K} \alpha_{k} \mathbf{e}_{u}^{(k)} ; \quad \mathbf{e}_{i}=\sum_{k=0}^{K} \alpha_{k} \mathbf{e}_{i}^{(k)}
$$

where $\alpha_{k} \geq 0$ denotes the importance of the $k$-th layer embedding in constituting the final embedding. In our experiments, we set $\alpha_{k}$ uniformly as $1 / (K+1)$.

The model prediction is defined as the inner product of user and item final representations:

$$
\hat{y}_{u i}=\mathbf{e}_{u}^{T} \mathbf{e}_{i}
$$

which is used as the ranking score for recommendation generation.


### 1.5 Matrix Form

Let the user-item interaction matrix be $\mathbf{R} \in \mathbb{R}^{M \times N}$ where $M$ and $N$ denote the number of users and items, respectively, and each entry $R_{ui}$ is 1 if $u$ has interacted with item $i$ otherwise 0. We then obtain the adjacency matrix of the user-item graph as

$$
\mathbf{A}=\left(\begin{array}{cc}
\mathbf{0} & \mathbf{R} \\
\mathbf{R}^{T} & \mathbf{0}
\end{array}\right)
$$

Let the 0-th layer embedding matrix be $\mathbf{E}^{(0)} \in \mathbb{R}^{(M+N) \times T}$, where $T$ is the embedding size. Then we can obtain the matrix equivalent form of LGC as:

$$
\mathbf{E}^{(k+1)}=\left(\mathbf{D}^{-\frac{1}{2}} \mathbf{A} \mathbf{D}^{-\frac{1}{2}}\right) \mathbf{E}^{(k)}
$$

where $\mathbf{D}$ is a $(M+N) \times(M+N)$ diagonal matrix, in which each entry $D_{ii}$ denotes the number of nonzero entries in the $i$-th row vector of the adjacency matrix $\mathbf{A}$ (also named as degree matrix). Lastly, we get the final embedding matrix used for model prediction as:

$$
\begin{aligned}
\mathbf{E} &=\alpha_{0} \mathbf{E}^{(0)}+\alpha_{1} \mathbf{E}^{(1)}+\alpha_{2} \mathbf{E}^{(2)}+\ldots+\alpha_{K} \mathbf{E}^{(K)} \\
&=\alpha_{0} \mathbf{E}^{(0)}+\alpha_{1} \tilde{\mathbf{A}} \mathbf{E}^{(0)}+\alpha_{2} \tilde{\mathbf{A}}^{2} \mathbf{E}^{(0)}+\ldots+\alpha_{K} \tilde{\mathbf{A}}^{K} \mathbf{E}^{(0)}
\end{aligned}
$$

where $\tilde{\mathbf{A}}=\mathbf{D}^{-\frac{1}{2}} \mathbf{A} \mathbf{D}^{-\frac{1}{2}}$ is the symmetrically normalized matrix.

### 1.6 Model Training

We employ the Bayesian Personalized Ranking (BPR) loss which is a pairwise loss that encourages the prediction of an observed entry to be higher than its unobserved counterparts:

$$
L_{B P R}=-\sum_{u=1}^{M} \sum_{i \in \mathcal{N}_{u}} \sum_{j \notin \mathcal{N}_{u}} \ln \sigma\left(\hat{y}_{u i}-\hat{y}_{u j}\right)+\lambda\left\|\mathbf{E}^{(0)}\right\|^{2}
$$

Where $\lambda$ controls the $L_2$ regularization strength. We employ the Adam optimizer and use it in a mini-batch manner.


## 2 TensorFlow implementation of LightGCN

### 2.1 Load and split data

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training.

In [3]:
# Load the outfit dataset (small CSV used for examples)
filepath = "datasets/csv/example_feature1.csv"
df = load_pandas_df(filepath=filepath, header=["UserId", "Weather", "Clothing", "Rating"])
df = df.rename(columns={'UserId': 'userID', 'Clothing': 'itemID', 'Rating': 'rating'})

df.head()

,userID,Weather,itemID,rating
0,1,Humid,Blazer,2.2
1,1,Rainy,Blazer,2.8
2,1,Sunny,Hoodie,2.5
3,1,Sunny,Jeans,3.8
4,2,Cloudy,Hoodie,4.1


In [4]:
# Stratified split using the outfit dataset column names
train, test = python_stratified_split(df, ratio=0.75, col_user="userID", col_item="itemID")
print(train)

    userID Weather             itemID  rating
0        1   Humid             Blazer     2.2
3        1   Sunny              Jeans     3.8
2        1   Sunny             Hoodie     2.5
10       2   Snowy             Hoodie     3.7
6        2   Humid           Cardigan     4.0
..     ...     ...                ...     ...
93      19   Windy              Jeans     3.9
91      19   Sunny  Long-sleeve shirt     3.5
98      20   Sunny             Chinos     3.3
99      20   Sunny            Joggers     2.4
97      20   Snowy             Hoodie     5.0

[77 rows x 4 columns]


### 2.2 Process data

`ImplicitCF` is a class that intializes and loads data for the training process. During the initialization of this class, user IDs and item IDs are reindexed, ratings greater than zero are converted into implicit positive interaction, and adjacency matrix $R$ of user-item graph is created. Some important methods of `ImplicitCF` are:

`get_norm_adj_mat`, load normalized adjacency matrix of user-item graph if it already exists in `adj_dir`, otherwise call `create_norm_adj_mat` to create the matrix and save the matrix if `adj_dir` is not `None`. This method will be called during the initialization process of LightGCN model.

`create_norm_adj_mat`, create normalized adjacency matrix of user-item graph by calculating $D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$, where $\mathbf{A}=\left(\begin{array}{cc}\mathbf{0} & \mathbf{R} \\ \mathbf{R}^{T} & \mathbf{0}\end{array}\right)$.

`train_loader`, generate a batch of training data — sample a batch of users and then sample one positive item and one negative item for each user. This method will be called before each epoch of the training process.


In [5]:
# Initialize ImplicitCF with the correct column names from the CSV
data = ImplicitCF(train=train, test=test, seed=SEED, col_user="userID", col_item="itemID", col_rating="rating")

### 2.3 Prepare hyper-parameters

Important parameters of `LightGCN` model are:

`data`, initialized LightGCNDataset object.

`epochs`, number of epochs for training.

`n_layers`, number of layers of the model.

`eval_epoch`, if it is not None, evaluation metrics will be calculated on test set every "eval_epoch" epochs. In this way, we can observe the effect of the model during the training process.

`top_k`, the number of items to be recommended for each user when calculating ranking metrics.

A complete list of parameters can be found in `yaml_file`. We use `prepare_hparams` to read the yaml file and prepare a full set of parameters for the model. Parameters passed as the function's parameters will overwrite yaml settings.

In [6]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

### 2.4 Create and train model

With data and parameters prepared, we can create the LightGCN model.

To train the model, we simply need to call the `fit()` method.

In [7]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2025-11-10 16:42:20.879046: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


In [8]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)0.1s: train loss = 0.63535 = (mf)0.63509 + (embed)0.00027
Epoch 2 (train)0.0s: train loss = 0.61728 = (mf)0.61700 + (embed)0.00028
Epoch 3 (train)0.0s: train loss = 0.58851 = (mf)0.58819 + (embed)0.00032
Epoch 4 (train)0.0s: train loss = 0.55006 = (mf)0.54970 + (embed)0.00035
Epoch 5 (train)0.0s + (eval)0.0s: train loss = 0.52343 = (mf)0.52304 + (embed)0.00039, recall = 0.76471, ndcg = 0.47382, precision = 0.10000, map = 0.35105
Epoch 6 (train)0.0s: train loss = 0.47255 = (mf)0.47211 + (embed)0.00044
Epoch 7 (train)0.0s: train loss = 0.43917 = (mf)0.43868 + (embed)0.00049
Epoch 8 (train)0.0s: train loss = 0.38981 = (mf)0.38925 + (embed)0.00056
Epoch 9 (train)0.0s: train loss = 0.35952 = (mf)0.35890 + (embed)0.00062
Epoch 10 (train)0.0s + (eval)0.0s: train loss = 0.30681 = (mf)0.30612 + (embed)0.00069, recall = 0.82353, ndcg = 0.48623, precision = 0.10588, map = 0.35016
Epoch 11 (train)0.0s: train loss = 0.25083 = (mf)0.25007 + (embed)0.00077
Epoch 12 (train)0.0s: train l

### 2.5 Recommendation and Evaluation

Recommendation and evaluation have been performed on the specified test set during training. After training, we can also use the model to perform recommendation and evalution on other data. Here we still use `test` as test data, but `test` can be replaced by other data with similar data structure.

#### 2.5.1 Recommendation

We can call `recommend_k_items` to recommend k items for each user passed in this function. We set `remove_seen=True` to remove the items already seen by the user. The function returns a dataframe, containing each user and top k items recommended to them and the corresponding ranking scores.

In [9]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

,userID,itemID,prediction
0,1,Chinos,1.824172
1,1,Sweatshirt,1.364667
2,1,Dress pants,0.821111
3,1,Cardigan,0.609889
4,1,Joggers,0.418790


#### 2.5.2 Evaluation

With `topk_scores` predicted by the model, we can evaluate how LightGCN performs on this test set.

In [10]:
eval_map = map(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.335609
NDCG:	0.477781
Precision@K:	0.105882
Recall@K:	0.823529
